# Search Assets

To search for data assets in Azure Purview using its REST API, you can use the search endpoint to query data assets based on specific criteria like name, classification, or other metadata.


## Code Breakdown:
### Authentication:
The get_access_token() function uses azure.identity.ClientSecretCredential to authenticate using client credentials and obtain an access token.
Search Data Assets:

The search_data_assets() function sends a POST request to the /search/advanced API endpoint. The keywords in the body represent the search term, and you can specify a limit for the number of results.

Handle Results:
The function processes the JSON response, extracts data asset details (like name, type, and GUID), and prints the result.


In [1]:
import requests
from azure.identity import ClientSecretCredential

In [2]:


# Azure AD and Purview settings
client_id = 'YOUR_CLIENT_ID' ## FROM APP REGISTRATION
client_secret = '<YOUR_CLIENT_SECRET>' ## FROM APP REGISTRATION SECRET VALUE
tenant_id = '<YOUR_TENANT_ID>'
purview_account_name = '<YOUR_PURVIEW_ACCOUNT_NAME>'
purview_endpoint = f'https://{purview_account_name}.purview.azure.com'

# Authenticate using Azure AD to get the access token
def get_access_token():
    credential = ClientSecretCredential(tenant_id, client_id, client_secret)
    token = credential.get_token('https://purview.azure.net/.default')
    return token.token

# Search data assets in Purview
def search_data_assets(search_term):
    url = f"{purview_endpoint}/catalog/api/atlas/v2/search/advanced"
    headers = {
        "Authorization": f"Bearer {get_access_token()}",
        "Content-Type": "application/json"
    }

    # Define the search body
    body = {
        "keywords": search_term,  # Search term or keywords
        "limit": 10               # Limit the number of results
    }

    response = requests.post(url, json=body, headers=headers)
    
    if response.status_code == 200:
        assets = response.json().get('entities', [])
        if assets:
            for asset in assets:
                print(f"Name: {asset['attributes']['name']}, Type: {asset['typeName']}, GUID: {asset['guid']}")
        else:
            print("No assets found.")
    else:
        print(f"Failed to search data assets: {response.text}")



In [ ]:

# Example usage
search_term = 'HR'  # You can search by keywords related to the data asset
search_data_assets(search_term)

### Sample Response

Name: sales-data-2023, Type: azure_blob_storage, GUID: 1234abcd-5678-efgh-ijkl-1234mnopqr
Name: sales-data-2022, Type: azure_sql_table, GUID: 5678mnop-1234-abcd-efgh-5678ijklqrst


## how to use API using MFA 


In [ ]:
import msal
import requests

# Azure AD App Registration settings
# Azure AD and Purview settings
client_id = '<YOUR_CLIENT_ID>'  # Azure AD App Registration Client ID
tenant_id = '<YOUR_TENANT_ID>'  # Azure AD Tenant ID
purview_account_name = '<YOUR_PURVIEW_ACCOUNT_NAME>'  # Purview account name
purview_endpoint = f"https://{purview_account_name}.purview.azure.com"


# Authentication authority and scope for Purview API
authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://purview.azure.net/.default"]  # API scope for Azure Purview

# Initialize MSAL public client application
app = msal.PublicClientApplication(client_id, authority=authority)

# Initiate the device code flow for interactive authentication
flow = app.initiate_device_flow(scopes=scope)
if "user_code" not in flow:
    raise Exception("Device flow initiation failed.")

# Display the code and URL for the user to authenticate
print(f"Go to {flow['verification_uri']} and enter the code: {flow['user_code']}")

# Wait for user to complete authentication and MFA
token_result = app.acquire_token_by_device_flow(flow)
if "access_token" not in token_result:
    raise Exception(f"Authentication failed: {token_result.get('error_description')}")

access_token = token_result['access_token']

# Set up the headers for the Purview Search API request
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Define the search query (customize as needed)
search_url = f"{purview_endpoint}/catalog/api/atlas/v2/search/advanced"
search_body = {
    "keywords": "HR",  # Replace with your search term
    "limit": 10  # Limit the number of search results
}

# Make the search API request
response = requests.post(search_url, json=search_body, headers=headers)

# Check and print the response
if response.status_code == 200:
    search_results = response.json().get('entities', [])
    for asset in search_results:
        print(f"Name: {asset['attributes']['name']}, Type: {asset['typeName']}, GUID: {asset['guid']}")
else:
    print(f"Search API call failed: {response.text}")

